In [69]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import FrenchStemmer
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/lahad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lahad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [70]:
# Prendre en main les données
data = pd.read_csv("../data/train.csv")
data.shape

(12473, 7)

Index(['doc_id', 'titre', 'type', 'difficulte', 'cout', 'ingredients',
       'recette'],
      dtype='object')

In [72]:


data['text'] = data['titre'] + " " + data['recette']
data = data[['text', 'type']]  
X = data['text']
y = data['type']


In [73]:
# Methode Baseline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

dummy = DummyClassifier(strategy='stratified')
dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

print(classification_report(y_test, y_pred))


                precision    recall  f1-score   support

       Dessert       0.31      0.31      0.31      1092
        Entrée       0.25      0.24      0.25       912
Plat principal       0.47      0.48      0.47      1738

      accuracy                           0.37      3742
     macro avg       0.34      0.34      0.34      3742
  weighted avg       0.37      0.37      0.37      3742



In [74]:
### Run2: TF-IDF + SVM

In [96]:
data = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")


data['text'] = data['titre'] + " " + data['recette']
data = data[['text', 'type']]  # Assurez-vous de remplacer 'label' par le nom de votre colonne de label
X = data['text']
y = data['type']



test['text'] = test['titre']+ " " + test['recette']
test = test[['text', 'type']]  # Assurez-vous de remplacer 'label' par le nom de votre colonne de label
X_testt = test['text']
y_testt = test['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_testt)

svm = SVC(kernel='linear', C=1, random_state=42)

svm.fit(X_train_tfidf, y_train)

y_pred = svm.predict(X_test_tfidf)

# Evaluation
accuracy = accuracy_score(y_testt, y_pred)
precision = precision_score(y_testt, y_pred, average='weighted')
recall = recall_score(y_testt, y_pred, average='weighted')
f1 = f1_score(y_testt, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred)
class_report = classification_report(y_testt, y_pred)

# Affichage des résultats
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Evaluation Metrics:
Accuracy: 0.8609510086455331
Precision: 0.8577441880603762
Recall: 0.8609510086455331
F1 Score: 0.8585459215001946
Confusion Matrix:
[[403   2   2]
 [  8 225 104]
 [  4  73 567]]
Classification Report:
                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98       407
        Entrée       0.75      0.67      0.71       337
Plat principal       0.84      0.88      0.86       644

      accuracy                           0.86      1388
     macro avg       0.85      0.85      0.85      1388
  weighted avg       0.86      0.86      0.86      1388



In [77]:
### Run3: Word2Vec

In [97]:
 #Modèle Word2Vec

test = pd.read_csv("../data/test.csv")

test['text'] = test['titre']+ " " + test['recette']
test = test[['text', 'type']]  # Assurez-vous de remplacer 'label' par le nom de votre colonne de label
X_testt = test['text']
y_testt = test['type']



from sklearn.svm import SVC
corpus = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, sg=1)

# Calcul des vecteurs de mots moyens pour chaque document
def document_vector(word2vec_model, doc):
    """Calculer le vecteur moyen pour un document en utilisant les vecteurs de mots Word2Vec"""
    # filtrer les mots absents dans le vocabulaire
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) != 0:
        return np.mean(word2vec_model.wv[doc], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

X_train_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_train])
X_test_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_testt])

svm_classifier = SVC(kernel='linear', C=10, random_state=42)
svm_classifier.fit(X_train_word2vec, y_train)


y_pred_word2vec = svm_classifier.predict(X_test_word2vec)

# Evaluation
accuracy = accuracy_score(y_testt, y_pred_word2vec)
precision = precision_score(y_testt, y_pred_word2vec, average='weighted')
recall = recall_score(y_testt, y_pred_word2vec, average='weighted')
f1 = f1_score(y_testt, y_pred_word2vec, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred_word2vec)
class_report = classification_report(y_testt, y_pred_word2vec)

# Affichage des résultats
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Evaluation Metrics:
Accuracy: 0.8378962536023055
Precision: 0.8336336642802019
Recall: 0.8378962536023055
F1 Score: 0.8291274228851641
Confusion Matrix:
[[404   2   1]
 [  9 179 149]
 [ 10  54 580]]
Classification Report:
                precision    recall  f1-score   support

       Dessert       0.96      0.99      0.97       407
        Entrée       0.76      0.53      0.63       337
Plat principal       0.79      0.90      0.84       644

      accuracy                           0.84      1388
     macro avg       0.84      0.81      0.81      1388
  weighted avg       0.83      0.84      0.83      1388



In [79]:
### Run4: CountVectorizer

In [98]:
## Modèle CountVectorizer
test = pd.read_csv("../data/test.csv")

test['text'] = test['titre']+ " " + test['recette']
test = test[['text', 'type']]  # Assurez-vous de remplacer 'label' par le nom de votre colonne de label
X_testt = test['text']
y_testt = test['type']

vectorizer = CountVectorizer()

X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_testt)

svm_classifier = SVC(random_state=42)
svm_classifier.fit(X_train_counts, y_train)

y_pred_counts = svm_classifier.predict(X_test_counts)



# Evaluation
accuracy = accuracy_score(y_testt, y_pred_counts)
precision = precision_score(y_testt, y_pred_counts, average='weighted')
recall = recall_score(y_testt, y_pred_counts, average='weighted')
f1 = f1_score(y_testt, y_pred_counts, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred_counts)
class_report = classification_report(y_testt, y_pred_counts)

# Affichage des résultats
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Evaluation Metrics:
Accuracy: 0.8566282420749279
Precision: 0.8528220369056105
Recall: 0.8566282420749279
F1 Score: 0.8517733884649409
Confusion Matrix:
[[404   1   2]
 [  9 208 120]
 [  8  59 577]]
Classification Report:
                precision    recall  f1-score   support

       Dessert       0.96      0.99      0.98       407
        Entrée       0.78      0.62      0.69       337
Plat principal       0.83      0.90      0.86       644

      accuracy                           0.86      1388
     macro avg       0.85      0.84      0.84      1388
  weighted avg       0.85      0.86      0.85      1388



In [82]:
######### TEST TF-IDF avec Nettotage de texte et Stemming   #########

In [83]:
stopwords_fr = set(stopwords.words('french'))
stemmer = FrenchStemmer()

def clean_text(text):
    # Supprimer les balises HTML
    text = re.sub(r'[^a-zA-ZÀ-ÿ]', ' ', text)
    text = text.lower()
    words = word_tokenize(text, language='french')
    words = [word for word in words if word not in stopwords_fr]
    words = [stemmer.stem(word) for word in words]
    cleaned_text = ' '.join(words)
    return cleaned_text



In [106]:
data = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# Prétraitement des données d'entraînement
data['text'] = data['titre']+ " " + data['recette']
data = data[['text', 'type']]
data['text'] = data['text'].apply(clean_text)
# data['text'] = data['text'].apply(stemmer)



test['text'] = test['titre']+ " " + test['recette']
test = test[['text', 'type']]
test['text'] = test['text'].apply(clean_text)
# test['text'] = test['text'].apply(stemmer)
X_testt = test['text']
y_testt = test['type']



# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['type'], test_size=0.3, random_state=42)

# Vecteur TF-IDF
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_testt)  # Utiliser transform, pas fit_transform

# Entraînement du modèle SVM
svm = SVC(kernel='rbf', C=10, random_state=42)
svm.fit(X_train_tfidf, y_train)

# Prédiction sur les données de test
y_pred = svm.predict(X_test_tfidf)


# Evaluation
accuracy = accuracy_score(y_testt, y_pred)
precision = precision_score(y_testt, y_pred, average='weighted')
recall = recall_score(y_testt, y_pred, average='weighted')
f1 = f1_score(y_testt, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred)
class_report = classification_report(y_testt, y_pred)

# Affichage des résultats
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)



In [100]:
# from sklearn.model_selection import GridSearchCV

# parameters = {'kernel':('linear', 'rbf'), 'C':[1,10]}
# svc = SVC(random_state=42)
# clf = GridSearchCV(svc, parameters)
# clf.fit(X_train_tfidf, y_train)
# #Print the best parameters
# print("Best parameters: ", clf.best_params_)
# # Use the best estimator to make predictions
# y_pred = clf.best_estimator_.predict(X_test_tfidf)

# # Evaluation
# print(classification_report(y_testt, y_pred))

In [101]:
####################################### Modèle CountVectorizer

data = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# Prétraitement des données d'entraînement
data['text'] = data['titre']+ " " + data['recette']
data = data[['text', 'type']]
data['text'] = data['text'].apply(clean_text)



test['text'] = test['titre']+ " " + test['recette']
test = test[['text', 'type']]
test['text'] = test['text'].apply(clean_text)
X_testt = test['text']
y_testt = test['type']

vectorizer = CountVectorizer()

X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_testt)

svm_classifier = SVC(kernel="rbf",random_state=42)
svm_classifier.fit(X_train_counts, y_train)

y_pred_counts = svm_classifier.predict(X_test_counts)


# Evaluation
accuracy = accuracy_score(y_testt, y_pred_counts)
precision = precision_score(y_testt, y_pred_counts, average='weighted')
recall = recall_score(y_testt, y_pred_counts, average='weighted')
f1 = f1_score(y_testt, y_pred_counts, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred_counts)
class_report = classification_report(y_testt, y_pred_counts)

# Affichage des résultats
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Evaluation Metrics:
Accuracy: 0.8739193083573487
Precision: 0.87126309253499
Recall: 0.8739193083573487
F1 Score: 0.8714596251334252
Confusion Matrix:
[[404   1   2]
 [  5 233  99]
 [  8  60 576]]
Classification Report:
                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98       407
        Entrée       0.79      0.69      0.74       337
Plat principal       0.85      0.89      0.87       644

      accuracy                           0.87      1388
     macro avg       0.87      0.86      0.86      1388
  weighted avg       0.87      0.87      0.87      1388



In [102]:
 #Modèle Word2Vec

data = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# Prétraitement des données d'entraînement
data['text'] = data['titre']+ " " + data['recette']
data = data[['text', 'type']]
data['text'] = data['text'].apply(clean_text)



test['text'] = test['titre']+ " " + test['recette']
test = test[['text', 'type']]
test['text'] = test['text'].apply(clean_text)
X_testt = test['text']
y_testt = test['type']



from sklearn.svm import SVC
corpus = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, sg=1)

# Calcul des vecteurs de mots moyens pour chaque document
def document_vector(word2vec_model, doc):
    """Calculer le vecteur moyen pour un document en utilisant les vecteurs de mots Word2Vec"""
    # filtrer les mots absents dans le vocabulaire
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) != 0:
        return np.mean(word2vec_model.wv[doc], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

X_train_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_train])
X_test_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_testt])

svm_classifier = SVC(kernel='linear', C=10, random_state=42)
svm_classifier.fit(X_train_word2vec, y_train)


y_pred_word2vec = svm_classifier.predict(X_test_word2vec)

# Evaluation
accuracy = accuracy_score(y_testt, y_pred_word2vec)
precision = precision_score(y_testt, y_pred_word2vec, average='weighted')
recall = recall_score(y_testt, y_pred_word2vec, average='weighted')
f1 = f1_score(y_testt, y_pred_word2vec, average='weighted')
conf_matrix = confusion_matrix(y_testt, y_pred_word2vec)
class_report = classification_report(y_testt, y_pred_word2vec)

# Affichage des résultats
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Evaluation Metrics:
Accuracy: 0.8566282420749279
Precision: 0.8535724704506641
Recall: 0.8566282420749279
F1 Score: 0.8539056442284936
Confusion Matrix:
[[402   3   2]
 [  4 219 114]
 [  5  71 568]]
Classification Report:
                precision    recall  f1-score   support

       Dessert       0.98      0.99      0.98       407
        Entrée       0.75      0.65      0.70       337
Plat principal       0.83      0.88      0.86       644

      accuracy                           0.86      1388
     macro avg       0.85      0.84      0.84      1388
  weighted avg       0.85      0.86      0.85      1388



In [104]:
####################### TEST Hyperpramètres #######################
